In [1]:
import pandas as pd
import numpy as np

c:\Users\bachk\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [21]:
data = pd.read_csv('../data/processed/real_estate/vic_rentals_all_enriched.csv')
data.shape

C:\Users\bachk\AppData\Local\Temp\ipykernel_20868\1274890271.py:1: DtypeWarning: Columns (18,23) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/processed/real_estate/vic_rentals_all_enriched.csv')


(17019, 64)

In [22]:
missing_list = [(col, data[col].isnull().sum()) for col in data.columns]
print(missing_list)

[('listing_id', 0), ('suburb', 0), ('postcode', 0), ('weekly_rent', 396), ('bond', 1106), ('available_date', 163), ('date_listed', 4), ('days_listed', 4), ('bedrooms', 143), ('bathrooms', 60), ('carspaces', 2265), ('property_type', 0), ('address', 113), ('lat', 4), ('lon', 4), ('photo_count', 4), ('video_count', 4), ('floorplans_count', 4), ('virtual_tour', 4), ('primary_type', 4), ('secondary_type', 4), ('agency', 5), ('agent_names', 321), ('land_area', 17017), ('num_metro_bus_stops', 0), ('num_metro_tram_stops', 0), ('num_metro_train_stops', 0), ('num_regional_bus_stops', 0), ('num_regional_train_stops', 0), ('num_schools_2km', 0), ('Median_age_persons', 4), ('Median_mortgage_repay_monthly', 4), ('Median_tot_prsnl_inc_weekly', 4), ('Median_rent_weekly', 4), ('Median_tot_fam_inc_weekly', 4), ('Average_num_psns_per_bedroom', 4), ('Median_tot_hhd_inc_weekly', 4), ('Average_household_size', 4), ('Owner occupied (%)', 4), ('Mortgage (%)', 4), ('Total rented (%)', 4), ('Other tenure (%)', 

In [ ]:
#Drop rows with small number of missing values
data = data.dropna(subset=['date_listed', 'lat', 'lon', 'bedrooms', 'bathrooms'])

In [4]:
data.shape

(16864, 64)

In [5]:
#Impute average rent for each combination of suburb, property_type, bedrooms, and bathrooms
# Create a lookup dictionary for average weekly rent
rent_lookup = (
    data.groupby(['suburb', 'property_type', 'bedrooms', 'bathrooms'])['weekly_rent']
    .mean()
    .round(0)   # optional: round to 0 decimals
    .to_dict()
)

In [6]:
#Impute missing weekly_rent values
for idx, row in data.iterrows():
    if pd.isnull(row['weekly_rent']):
        key = (row['suburb'], row['property_type'], row['bedrooms'], row['bathrooms'])
        if key in rent_lookup:  # only impute if lookup exists
            data.at[idx, 'weekly_rent'] = rent_lookup[key]

In [7]:
#Check for missing values again
missing_list = [(col, data[col].isnull().sum()) for col in data.columns]
print(missing_list)

[('listing_id', 0), ('suburb', 0), ('postcode', 0), ('weekly_rent', 113), ('bond', 1080), ('available_date', 153), ('date_listed', 0), ('days_listed', 0), ('bedrooms', 0), ('bathrooms', 0), ('carspaces', 2175), ('property_type', 0), ('address', 107), ('lat', 0), ('lon', 0), ('photo_count', 0), ('video_count', 0), ('floorplans_count', 0), ('virtual_tour', 0), ('primary_type', 0), ('secondary_type', 0), ('agency', 1), ('agent_names', 301), ('land_area', 16862), ('num_metro_bus_stops', 0), ('num_metro_tram_stops', 0), ('num_metro_train_stops', 0), ('num_regional_bus_stops', 0), ('num_regional_train_stops', 0), ('num_schools_2km', 0), ('Median_age_persons', 0), ('Median_mortgage_repay_monthly', 0), ('Median_tot_prsnl_inc_weekly', 0), ('Median_rent_weekly', 0), ('Median_tot_fam_inc_weekly', 0), ('Average_num_psns_per_bedroom', 0), ('Median_tot_hhd_inc_weekly', 0), ('Average_household_size', 0), ('Owner occupied (%)', 0), ('Mortgage (%)', 0), ('Total rented (%)', 0), ('Other tenure (%)', 0),

In [ ]:
#Impute average rent with relaxed contraints (without bathrooms and suburb) to fill rest of missing values
#Create a lookup dictionary for average weekly rent
rent_lookup = (
    data.groupby(['property_type', 'bedrooms'])['weekly_rent']
    .mean()
    .round(0)   # optional: round to 0 decimals
    .to_dict()
)

In [9]:
#Impute missing weekly_rent values
for idx, row in data.iterrows():
    if pd.isnull(row['weekly_rent']):
        key = (row['property_type'], row['bedrooms'])
        if key in rent_lookup:  # only impute if lookup exists
            data.at[idx, 'weekly_rent'] = rent_lookup[key]

In [10]:
#Check for missing values again
missing_list = [(col, data[col].isnull().sum()) for col in data.columns]
print(missing_list)

[('listing_id', 0), ('suburb', 0), ('postcode', 0), ('weekly_rent', 2), ('bond', 1080), ('available_date', 153), ('date_listed', 0), ('days_listed', 0), ('bedrooms', 0), ('bathrooms', 0), ('carspaces', 2175), ('property_type', 0), ('address', 107), ('lat', 0), ('lon', 0), ('photo_count', 0), ('video_count', 0), ('floorplans_count', 0), ('virtual_tour', 0), ('primary_type', 0), ('secondary_type', 0), ('agency', 1), ('agent_names', 301), ('land_area', 16862), ('num_metro_bus_stops', 0), ('num_metro_tram_stops', 0), ('num_metro_train_stops', 0), ('num_regional_bus_stops', 0), ('num_regional_train_stops', 0), ('num_schools_2km', 0), ('Median_age_persons', 0), ('Median_mortgage_repay_monthly', 0), ('Median_tot_prsnl_inc_weekly', 0), ('Median_rent_weekly', 0), ('Median_tot_fam_inc_weekly', 0), ('Average_num_psns_per_bedroom', 0), ('Median_tot_hhd_inc_weekly', 0), ('Average_household_size', 0), ('Owner occupied (%)', 0), ('Mortgage (%)', 0), ('Total rented (%)', 0), ('Other tenure (%)', 0), (

In [11]:
data = data.dropna(subset=['weekly_rent'])

In [13]:
#Impute average carspaces for each combination of suburb, property_type, bedrooms, and bathrooms
# Create a lookup dictionary for average carspaces
carspace_lookup = (
    data.groupby(['suburb', 'property_type', 'bedrooms', 'bathrooms'])['carspaces']
    .mean()
    .round(0)   # optional: round to 0 decimals
    .to_dict()
)

In [14]:
#Impute missing carspaces values
for idx, row in data.iterrows():
    if pd.isnull(row['carspaces']):
        key = (row['suburb'], row['property_type'], row['bedrooms'], row['bathrooms'])
        if key in carspace_lookup:  # only impute if lookup exists
            data.at[idx, 'carspaces'] = carspace_lookup[key]

In [15]:
#Check for missing values again
missing_list = [(col, data[col].isnull().sum()) for col in data.columns]
print(missing_list)

[('listing_id', 0), ('suburb', 0), ('postcode', 0), ('weekly_rent', 0), ('bond', 1080), ('available_date', 153), ('date_listed', 0), ('days_listed', 0), ('bedrooms', 0), ('bathrooms', 0), ('carspaces', 500), ('property_type', 0), ('address', 107), ('lat', 0), ('lon', 0), ('photo_count', 0), ('video_count', 0), ('floorplans_count', 0), ('virtual_tour', 0), ('primary_type', 0), ('secondary_type', 0), ('agency', 1), ('agent_names', 301), ('land_area', 16860), ('num_metro_bus_stops', 0), ('num_metro_tram_stops', 0), ('num_metro_train_stops', 0), ('num_regional_bus_stops', 0), ('num_regional_train_stops', 0), ('num_schools_2km', 0), ('Median_age_persons', 0), ('Median_mortgage_repay_monthly', 0), ('Median_tot_prsnl_inc_weekly', 0), ('Median_rent_weekly', 0), ('Median_tot_fam_inc_weekly', 0), ('Average_num_psns_per_bedroom', 0), ('Median_tot_hhd_inc_weekly', 0), ('Average_household_size', 0), ('Owner occupied (%)', 0), ('Mortgage (%)', 0), ('Total rented (%)', 0), ('Other tenure (%)', 0), ('

In [16]:
#Impute average carspaces with relaxed contraints (without bathrooms) to fill rest of missing values
# Create a lookup dictionary for average carspaces
carspace_lookup = (
    data.groupby(['property_type', 'bedrooms'])['carspaces']
    .mean()
    .round(0)   # optional: round to 0 decimals
    .to_dict()
)

In [17]:
#Impute missing carspaces values
for idx, row in data.iterrows():
    if pd.isnull(row['carspaces']):
        key = ( row['property_type'], row['bedrooms'])
        if key in carspace_lookup:  # only impute if lookup exists
            data.at[idx, 'carspaces'] = carspace_lookup[key]

In [18]:
#Check for missing values again
missing_list = [(col, data[col].isnull().sum()) for col in data.columns]
print(missing_list)

[('listing_id', 0), ('suburb', 0), ('postcode', 0), ('weekly_rent', 0), ('bond', 1080), ('available_date', 153), ('date_listed', 0), ('days_listed', 0), ('bedrooms', 0), ('bathrooms', 0), ('carspaces', 10), ('property_type', 0), ('address', 107), ('lat', 0), ('lon', 0), ('photo_count', 0), ('video_count', 0), ('floorplans_count', 0), ('virtual_tour', 0), ('primary_type', 0), ('secondary_type', 0), ('agency', 1), ('agent_names', 301), ('land_area', 16860), ('num_metro_bus_stops', 0), ('num_metro_tram_stops', 0), ('num_metro_train_stops', 0), ('num_regional_bus_stops', 0), ('num_regional_train_stops', 0), ('num_schools_2km', 0), ('Median_age_persons', 0), ('Median_mortgage_repay_monthly', 0), ('Median_tot_prsnl_inc_weekly', 0), ('Median_rent_weekly', 0), ('Median_tot_fam_inc_weekly', 0), ('Average_num_psns_per_bedroom', 0), ('Median_tot_hhd_inc_weekly', 0), ('Average_household_size', 0), ('Owner occupied (%)', 0), ('Mortgage (%)', 0), ('Total rented (%)', 0), ('Other tenure (%)', 0), ('U

In [23]:
data = data.dropna(subset=['carspaces'])